In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('gta_sa_vehicle_stats.csv')
pd.set_option('display.max_columns', None)
df

In [ ]:
df.columns = [c.lower().replace(' ', '_') for c in df.columns]

In [ ]:
df

In [ ]:
df = df.rename(columns={
    "center_of_mass": "center_of_mass_x",
    "center_of_mass.1": "center_of_mass_y",
    "center_of_mass.2": "center_of_mass_z",
    "center_of_mass.3": "center_of_mass_%submerged",
    "traction": "traction_multiplier",
    "traction.1": "traction_loss",
    "traction.2": "traction_bias(%)",
    "transmission": "#_of_gears",
    "transmission.1": "max_velocity(km/h)",
    "transmission.2": "acceleration(ms^2)",
    "transmission.3": "interia",
    "transmission.4": "drive_type",
    "transmission.5": "engine_type",
    "brakes": "brakes_deceleration(ms^2)",
    "brakes.1": "brakes_bias(%)",
    "brakes.2": "abs",
    "suspension": "suspension_force_level",
    "suspension.1": "suspension_damping_level",
    "suspension.2": "suspension_high_speed_com_damping",
    "suspension_lines": "suspension_lines_upper_limit",
    "suspension_lines.1": "suspension_lines_lower_limit",
    "suspension_lines.2": "suspension_lines_bias_between_front_and_rear",
    "lights": "lights_front",
    "lights.1": "lights_rear",
    "lights.2": "lights_anim_group",

})

In [ ]:
df = df.iloc[1:].reset_index(drop=True)
df

Save this cleared version

In [ ]:
#df.to_csv('gta_sa_vehicle_stats_cleared.csv', index=False)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df['lights_anim_group'] = df['lights_anim_group'].fillna(0)

In [ ]:
df.isnull().sum().sum()

In [ ]:
df['max_velocity(km/h)'] = df['max_velocity(km/h)'].replace('300.0', '200.0')

In [ ]:
# Nowe wartości jako słownik
correct_values = {
    'STRETCH': '140.0',
    'BRAVURA': '144.0',
    'LINERUN': '160.0',
    'TRASH': '170.0',
    'MANANA': '160.0',
    'DUMPER': '110.0',
    'FIRETRUK': '170.0',
    'URANUS': '180.0',
    'PEREN': '150.0',
    'SENTINEL': '165.0'
}

# Aktualizacja kolumny 'max_velocity(km/h)' w df
for vehicle, speed in correct_values.items():
    df.loc[df['identifier'] == vehicle, 'max_velocity(km/h)'] = speed

In [ ]:
df

In [ ]:
#df.to_csv('gta_sa_vehicle_stats_ready.csv', index=False)

TERAZ DZIEJE SIE EDA

In [ ]:
df.dtypes

In [ ]:
for c in df.columns:
    if c not in ['identifier', 'drive_type', 'engine_type', 'model_flags', 'handling_flags', 'lights_front', 'lights_rear']:
        df[c] = pd.to_numeric(df[c])

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

In [ ]:
df.select_dtypes(include='object')

In [ ]:
df.select_dtypes(include='object').nunique()

In [ ]:
for c in df.select_dtypes(include='object').columns:
    if c not in ['identifier', 'model_flags', 'handling_flags']:
        print(f'Unique Values in "{c}":', df[c].unique())

In [ ]:
df['drive_type'] = df['drive_type'].replace('R', 'Rear')
df['engine_type'] = df['engine_type'].replace('D', 'Diesel')

In [ ]:
for c in ['drive_type', 'engine_type']:
    print(f'Unique Values in {c}:', df[c].unique())

In [ ]:
df['drive_type'] = df['drive_type'].astype('category')
df['engine_type'] = df['engine_type'].astype('category')

TYPY ZROBIONE - BRAKUJE SWIATEL ALE HUJ - JAZDA Z WYKRESAMI

In [ ]:
df.select_dtypes(include='category')

STATYSKYKI OPISOWE I ROZKŁADY

In [ ]:
#informacje ogólne
df.info()

In [ ]:
df.describe().T

In [ ]:
#histogramy - rozkład najważniejszych cech

In [ ]:
df.columns

In [ ]:
features = ['mass_(kg)', 'acceleration(ms^2)', 'max_velocity(km/h)', 'monetary_value_($)']

plt.figure(figsize=(16, 10))
for i, c in enumerate(features, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[c], bins=25, kde=True)
    plt.title(f'Distribution of {c}')

In [ ]:
#rozkład typów napędu

In [ ]:
sns.countplot(x=df['drive_type'], order=['Rear', 'Front', '4'])
plt.title('Drive Type Distribution')

In [ ]:
sns.countplot(x=df['engine_type'], order=['Petrol', 'Diesel', 'Electric'])
plt.title('Engine Type Distribution')

In [ ]:
#boxplot masa vs przyspieszenie

In [ ]:
sns.boxplot(x=df['drive_type'], y=df['acceleration(ms^2)'], order=['Rear', 'Front', '4'])
plt.title('Acceleration by Drive Type')

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(x='mass_(kg)', y='acceleration(ms^2)', hue='drive_type', data=df)
plt.title('Mass vs Acceleration by Drive Type')

In [ ]:
df_filtered = df[(df['mass_(kg)'] <= 10000) & (df['acceleration(ms^2)'] <= 35)]

plt.figure(figsize=(10, 8))
sns.scatterplot(x='mass_(kg)', y='acceleration(ms^2)', hue='drive_type', data=df_filtered)
plt.title('Mass vs Acceleration by Drive Type')

In [ ]:
#przyczepność vs rozkład hamulców

In [ ]:
sns.scatterplot(x='traction_multiplier', y='brakes_bias(%)', data=df)
plt.title('Traction vs Brake bias')

In [ ]:
#najważniejsze cechy techniczne

In [ ]:
features1 = ['mass_(kg)', 'acceleration(ms^2)', 'max_velocity(km/h)',
            'traction_multiplier', 'brakes_deceleration(ms^2)', 'collision_damage_multiplier']

sns.pairplot(df[features1])

TO BYŁA JEDNA PORCJA, TERAZ JAZDA DALEJ

In [ ]:
#top/bottom:

In [ ]:
df.columns

In [ ]:
top_speed = df.sort_values(by='max_velocity(km/h)', ascending=False).head(15)
top_speed

In [ ]:
plt.figure(figsize=(16, 10))
sns.barplot(y='identifier', x='max_velocity(km/h)', hue='max_velocity(km/h)', data=top_speed, palette='bright')
plt.title('Top 15 Fastest Cars')

In [ ]:
slowest = df.sort_values(by='max_velocity(km/h)', ascending=True).head(15)
slowest

In [ ]:
plt.figure(figsize=(16, 10))
sns.barplot(y='identifier', x='max_velocity(km/h)', hue='max_velocity(km/h)', data=slowest, palette='bright')
plt.title('Top 15 Slowest Cars')

In [ ]:
acceleration = df.sort_values(by='acceleration(ms^2)', ascending=False).head(10)
plt.figure(figsize=(16, 10))
sns.barplot(y='identifier', x='acceleration(ms^2)', hue='acceleration(ms^2)', data=acceleration, palette='deep')
plt.title('Top 10 Acceleration')

In [ ]:
heaviest = df.sort_values(by='mass_(kg)', ascending=False).head(10)
plt.figure(figsize=(16, 10))
sns.barplot(x='identifier', y='mass_(kg)', hue='mass_(kg)', data=heaviest)
plt.title('Top 10 Heaviest Vehicles')

In [ ]:
toughest = df.sort_values(by='collision_damage_multiplier', ascending=True).head(10)
plt.figure(figsize=(16, 10))
sns.barplot(x='identifier', y='collision_damage_multiplier', hue='collision_damage_multiplier', data=toughest, palette='flare')
plt.title('Top 10 Most Damage-Resistant Vehicles (Lower = Tougher)')

Topki zrobione, czas na zależności i wykresy 2D

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(16, 10))
sns.regplot(x='monetary_value_($)', y='max_velocity(km/h)', data=df)
plt.title('Car Price vs Top Speed')

In [ ]:
plt.figure(figsize=(16, 10))
sns.regplot(x='monetary_value_($)', y='acceleration(ms^2)', data=df)
plt.title('Car Price vs Acceleration')

In [ ]:
plt.figure(figsize=(16, 10))
sns.regplot(x='mass_(kg)', y='collision_damage_multiplier', data=df)
plt.title('Are heavier vehicles more durable?')

In [ ]:
plt.figure(figsize=(16, 10))
sns.regplot(x='max_velocity(km/h)', y='acceleration(ms^2)', data=df)
plt.title('Top Speed vs Acceleration')